In [18]:
df = spark.read.option("multiline", "true").json("Files/bing-latest-news.json")

StatementMeta(, 5a2e3e97-28bd-481b-bbb4-947e30def9c4, 20, Finished, Available, Finished)

In [19]:
display(df)

StatementMeta(, 5a2e3e97-28bd-481b-bbb4-947e30def9c4, 21, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, bb8310b3-40ff-43db-abe3-f16a13c95207)

In [20]:
df = df.select("value")

StatementMeta(, 5a2e3e97-28bd-481b-bbb4-947e30def9c4, 22, Finished, Available, Finished)

In [21]:
from pyspark.sql.functions import explode
df_exploded = df.select(explode(df["value"]).alias("json_object"))

StatementMeta(, 5a2e3e97-28bd-481b-bbb4-947e30def9c4, 23, Finished, Available, Finished)

In [22]:
display(df_exploded)

StatementMeta(, 5a2e3e97-28bd-481b-bbb4-947e30def9c4, 24, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 40cd184f-2536-46d1-9a32-9a9a077d3d1c)

In [23]:
json_list = df_exploded.toJSON().collect()

StatementMeta(, 5a2e3e97-28bd-481b-bbb4-947e30def9c4, 25, Finished, Available, Finished)

In [24]:
print(json_list[1])

StatementMeta(, 5a2e3e97-28bd-481b-bbb4-947e30def9c4, 26, Finished, Available, Finished)

{"json_object":{"about":[{"name":"Arsenal F.C.","readLink":"https://api.bing.microsoft.com/api/v7/entities/8b3b4135-cb8e-ded8-10ce-8f130d64b863"},{"name":"Paris Saint-Germain F.C.","readLink":"https://api.bing.microsoft.com/api/v7/entities/67266d4d-2667-f33f-7d3e-ae1225abff86"},{"name":"UEFA Champions League","readLink":"https://api.bing.microsoft.com/api/v7/entities/eef12ef2-8dba-1e7b-b5c7-87f4be811495"},{"name":"Mikel Merino","readLink":"https://api.bing.microsoft.com/api/v7/entities/e2365c49-c2b6-aff5-437d-72f2ca96a378"},{"name":"Crystal Palace F.C.","readLink":"https://api.bing.microsoft.com/api/v7/entities/9ace019e-6c58-d201-31b2-73cc6388867a"},{"name":"Emirates Stadium","readLink":"https://api.bing.microsoft.com/api/v7/entities/1b347604-a9b2-7364-f2f3-9a382ca644ab"},{"name":"Ben White","readLink":"https://api.bing.microsoft.com/api/v7/entities/2406a76d-2dfb-cd9f-4862-4e5e21d10af4"},{"name":"Mikel Arteta","readLink":"https://api.bing.microsoft.com/api/v7/entities/46cbb218-c295-c25

In [25]:
import json

news_json = json.loads(json_list[1]) #Converting the JSON string to a JSON dictionary
#print(news_json)
#print(news_json["json_object"]["name"])
#print(news_json["json_object"]["category"])

StatementMeta(, 5a2e3e97-28bd-481b-bbb4-947e30def9c4, 27, Finished, Available, Finished)

In [26]:
title = []
description = []
category = []
url =[]
image = []
provider = []
datePublished =[]


# Process each JSON object in the list
for json_str in json_list:
    try:
        # Parse the JSON string into a dictionary
        article = json.loads(json_str)

        if article["json_object"].get("category") and article["json_object"].get("image", {}).get("thumbnail", {}).get("contentUrl"):
        
            #Extract information from the dictionary
            title.append(article["json_object"]["name"])
            description.append(article["json_object"]["description"])
            category.append(article["json_object"]["category"])
            url.append(article["json_object"]["url"])
            image.append(article["json_object"]["image"]["thumbnail"]["contentUrl"])
            provider.append(article["json_object"]["provider"][0]['name'])
            datePublished.append(article["json_object"]["datePublished"])

    except Exception as e:
        print(f"Error processing JSON object: {e}")

StatementMeta(, 5a2e3e97-28bd-481b-bbb4-947e30def9c4, 28, Finished, Available, Finished)

In [27]:
from pyspark.sql.types import StructType, StructField, StringType


# Combine the lists
data = list(zip(title,description,category,url,image,provider,datePublished))

# Define schema
schema = StructType([
    StructField("title", StringType(), True),
    StructField("description", StringType(), True),
    StructField("category", StringType(), True),
    StructField("url", StringType(), True),
    StructField("image", StringType(), True),
    StructField("provider", StringType(), True),
    StructField("datePublished", StringType(), True)
])

# Create DataFrame
df_cleaned = spark.createDataFrame(data, schema=schema)

StatementMeta(, 5a2e3e97-28bd-481b-bbb4-947e30def9c4, 29, Finished, Available, Finished)

In [28]:
display(df_cleaned)

StatementMeta(, 5a2e3e97-28bd-481b-bbb4-947e30def9c4, 30, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, b4ee87f1-34b0-45f9-8ba0-8afe9286604b)

In [29]:
from pyspark.sql.functions import to_date, date_format

df_cleaned_final = df_cleaned.withColumn("datePublished", date_format(to_date("datePublished"), "dd-MMM-yyyy"))

StatementMeta(, 5a2e3e97-28bd-481b-bbb4-947e30def9c4, 31, Finished, Available, Finished)

In [31]:
from pyspark.sql.utils import AnalysisException

try:

    table_name = 'bing_lake_db.tbl_latest_news'

    df_cleaned_final.write.format("delta").saveAsTable(table_name)

except AnalysisException:

    print("Table Already Exists")

    df_cleaned_final.createOrReplaceTempView("vw_df_cleaned_final")

    spark.sql(f"""  MERGE INTO {table_name} target_table
                    USING vw_df_cleaned_final source_view

                    ON source_view.url = target_table.url
                    
                    WHEN MATCHED AND 
                    source_view.title <> target_table.title OR
                    source_view.description <> target_table.description OR
                    source_view.category <> target_table.category OR
                    source_view.image <> target_table.image OR
                    source_view.provider <> target_table.provider OR
                    source_view.datePublished <> target_table.datePublished   

                    THEN UPDATE SET *

                    WHEN NOT MATCHED THEN INSERT *

                """)

StatementMeta(, 5a2e3e97-28bd-481b-bbb4-947e30def9c4, 33, Finished, Available, Finished)